# Install required packages
- Torch 2.1.2 with Cuda 12.1
- Torchvision 0.16.2 with Cuda 12.1
- MMCV 2.1.0 with cuda 12.1
- MMdetection 3.0.3
- TIMM

# Download weights file
- Vision Transformer huge (630M) / 16

In [ ]:
!pip install torch==2.1.2+cu121 torchvision==0.16.2+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.1/index.html
!pip install -U openmim
!mim install mmdet
!pip install timm

!wget s3.amazonaws.com/dataforgood-fb-data/forests/v1/models/saved_checkpoints/SSLhuge_satellite.pth

In [ ]:
# Check to make sure that all imports are correct
import shutil
import os

import torch
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

import mmcv
print('mmcv', mmcv.__version__)

import mmengine
print(mmengine.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Download and unzip the github repository

In [ ]:
!wget https://github.com/wri/mmdetection-satellite-dinov2/archive/refs/heads/main.zip
!unzip -o main.zip

# Mount the google drive and copy the training data, move the backbone weights

In [ ]:
# Mount google drive (alternatively copy training data from AWS)
import os
import shutil
from google.colab import drive
drive.mount('/content/drive')
!unzip drive/MyDrive/coco/tree-may-3.zip -d mmdetection-satellite-dinov2-main/data/coco/
shutil.move("SSLhuge_satellite.pth", "mmdetection-satellite-dinov2-main/models/SSLhuge_satellite.pth")

# Copy edited files to MMDET sys.path location

In [ ]:
import mmdet
import shutil
mmdet_location = "/".join(mmdet.__file__.split("/")[:-1]) + "/"
base_location = '/content/mmdetection-satellite-dinov2-main/mmdet/'

files_to_move = ['/models/backbones/SSLVisionTransformer.py',
                 '/models/backbones/vit_rvsa_mtp_branches.py',
                 'models/backbones/__init__.py',
                 'models/necks/__init__.py',
                 'models/dense_heads/__init__.py',
                 'models/necks/fpn.py',
                 'models/dense_heads/crpn_head.py',
                 'models/dense_heads/cascade_rpn_head.py',
                 'models/dense_heads/detr_head.py',
                 'models/dense_heads/dino_head.py',
                 'models/task_modules/assigners/__init__.py',
                 'models/task_modules/assigners/dynamic_assigner.py',
                 'models/task_modules/assigners/ranking_assigner.py',
                 'models/task_modules/assigners/iou2d_calculator.py',
                 'models/task_modules/assigners/hierarchical_assigner.py',
                 'models/layers/transformer/dino_layers.py'
                ]


for file in files_to_move:
    shutil.copy(f"{base_location}{file}",
                f'{mmdet_location}{file}')

# Train model

In [ ]:
import sys
import os.path as osp
sys.path.append(osp.abspath('mmdetection-satellite-dinov2-main/'))

from mmdet.apis import init_detector
from mmengine.runner import Runner
from mmengine.config import Config, DictAction

config='mmdetection-satellite-dinov2-main/projects/ViTDet/configs/vitdet-codetr.py'
checkpoint = 'mmdetection-satellite-dinov2-main/models/SSLhuge_satellite.pth'
cfg = Config.fromfile(config)
cfg['model']['backbone']['init_cfg']['checkpoint'] = checkpoint
cfg['resume'] = True
cfg['train_cfg']['max_epochs'] = 80
cfg.work_dir = osp.join('./work_dirs', osp.splitext(osp.basename(config))[0] + '-codetr')

runner = Runner.from_cfg(cfg)
runner.train()